In [86]:
import pandas as pd
import numpy as np
import tensorflow_hub as hub
# import tensorflow_datasets as tfds
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn import preprocessing , neighbors ,svm
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [87]:
xss = pd.read_csv('XSSDataSets/XSSTesting.csv')

In [88]:
xss.head


<bound method NDFrame.head of        Contains &lt  ScripTag  Readable  Contains "><  Contains '><  \
0                 0         1         0             0             0   
1                 0         1         0             0             0   
2                 0         0         1             0             0   
3                 0         1         1             0             0   
4                 0         1         0             0             0   
...             ...       ...       ...           ...           ...   
24091             0         0         1             0             0   
24092             0         0         1             0             0   
24093             0         0         1             0             0   
24094             0         0         1             0             0   
24095             0         0         1             0             0   

       Contains And  Contains Percentage  Contains Slash  Contains BackSlash  \
0                 0                  

In [89]:
def scale_data(df, column_list):
    for column in column_list:
        df[column] = df[column].astype('float')
        encoder = preprocessing.StandardScaler()
        df[column] = encoder.fit_transform(df[column].values.reshape(-1,1))
        print("The ",column, "is encoded")
    return(df)
  
def encode_categorical(df, column_list):
    for column in column_list:
        df[column] = df[column].astype('str')
        encoder = preprocessing.LabelEncoder()
        encoded_list = encoder.fit_transform(df[column])
#         print(encoded_list)
#         print(len(encoded_list))
        encoded_series = pd.Series(encoded_list)
        df[column] = encoded_series
        print("The ", column, "is encoded ")
    return(df)

df = encode_categorical(xss,['Class'])

The  Class is encoded 


In [90]:
X = np.array(xss.drop(labels=['Class'],axis=1)).reshape(len(xss),1,65)
print(len(X))

y = np.array(df['Class'])
print(y)

24096
[1 1 1 ... 0 0 0]


In [91]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [92]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [93]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [94]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(20, activation='relu', input_shape= (None,65)))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, None, 20)          1320      
_________________________________________________________________
dense_13 (Dense)             (None, None, 16)          336       
_________________________________________________________________
dense_14 (Dense)             (None, None, 1)           17        
Total params: 1,673
Trainable params: 1,673
Non-trainable params: 0
_________________________________________________________________


In [95]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [96]:
history = model.fit(train_dataset,
                    epochs=20,
                    verbose=1)

Epoch 1/20
302/302 [==============================] - 1s 1ms/step - loss: 0.5769 - accuracy: 0.9535: 0s - loss: 0.5889 - accuracy: 0.
Epoch 2/20
302/302 [==============================] - 0s 1ms/step - loss: 0.5386 - accuracy: 0.9941
Epoch 3/20
302/302 [==============================] - 0s 1ms/step - loss: 0.5369 - accuracy: 0.9985
Epoch 4/20
302/302 [==============================] - 1s 2ms/step - loss: 0.5364 - accuracy: 0.9993
Epoch 5/20
302/302 [==============================] - 0s 1ms/step - loss: 0.5362 - accuracy: 0.9993: 0s - loss: 0.5360 - accuracy: 0.
Epoch 6/20
302/302 [==============================] - 0s 1ms/step - loss: 0.5362 - accuracy: 0.9997
Epoch 7/20
302/302 [==============================] - 0s 1ms/step - loss: 0.5361 - accuracy: 0.9997
Epoch 8/20
302/302 [==============================] - 0s 1ms/step - loss: 0.5361 - accuracy: 0.9997
Epoch 9/20
302/302 [==============================] - 0s 1ms/step - loss: 0.5361 - accuracy: 0.9997
Epoch 10/20
302/302 [===========

In [104]:
model.evaluate(test_dataset)

model.save("xss.h5")

76/76 [==============================] - 0s 1ms/step - loss: 0.5343 - accuracy: 0.9990


In [105]:
def cal_keyword(eval_string,comparison_s):
    for i in range(0,len(eval_string)):
        if i+len(comparison_s)<len(eval_string):
            if eval_string[i]==comparison_s[0]:
                
                j = i
               
                for q in range(0,len(comparison_s)):
                    if eval_string[j]!=comparison_s[q]:
                        break
                    
                    j+=1
                   
                if (j-i)==len(comparison_s):
                    return 1.0
    
    return 0.0
def letter_ratio(eval_s):
    count = 0.0
    for i in range(0,len(eval_s)):
        if ord(eval_s[i].lower()) > 96 and ord(eval_s[i].lower()) < 123:
            count+=1.0
    return count / len(eval_s)
def number_ratio(eval_s):
    count = 0.0
    for i in range(0,len(eval_s)):
        if ord(eval_s[i].lower()) > 47 and ord(eval_s[i].lower()) < 58:
            count+=1.0
    return count / len(eval_s)
def symbol_ratio(eval_s):
    count = 0.0
    for i in range(0,len(eval_s)):
        if ord(eval_s[i].lower()) > 32 and ord(eval_s[i].lower()) < 48:
            count+=1.0
        if ord(eval_s[i].lower()) > 37 and ord(eval_s[i].lower()) < 65:
            count+=1.0
        if ord(eval_s[i].lower()) > 90 and ord(eval_s[i].lower()) < 97:
            count+=1.0
        if ord(eval_s[i].lower()) > 122 and ord(eval_s[i].lower()) < 127:
            count+=1.0
    return count / len(eval_s)

In [132]:
evaluation_string="<video src=1 href=1 onerror=\"javascript:alert(1)\"></video>"
new_model=tf.keras.models.load_model("xss.h5")
payload={}
payload['Contains &lt']= cal_keyword(evaluation_string,"&lt")
payload['ScriptTag']=cal_keyword(evaluation_string,"script")
payload['Readable']=1
payload['Contains "><']=cal_keyword(evaluation_string,"\"><")
payload['Contains \'><']=cal_keyword(evaluation_string,"\'><")
payload['Contains And']=cal_keyword(evaluation_string,"&")
payload['Contains Percentage']=cal_keyword(evaluation_string,"%")
payload['Contains Slash']=cal_keyword(evaluation_string,"/")
payload['Contains BackSlash']=cal_keyword(evaluation_string,"\\")
payload['Contains Plus']=cal_keyword(evaluation_string,"+")
payload['Contains Document']=cal_keyword(evaluation_string,"document")
payload['Contains Window']=cal_keyword(evaluation_string,"window")
payload['Contains Onload']=cal_keyword(evaluation_string,"onload")
payload['Contains Onerror']=cal_keyword(evaluation_string,"onerror")
payload['Contains DIV']=cal_keyword(evaluation_string,"div")
payload['Contains iframe']=cal_keyword(evaluation_string,"iframe")
payload['Contains img']=cal_keyword(evaluation_string,"img")
payload['Contains SRC']=cal_keyword(evaluation_string,"src")
payload['Contains Var']=cal_keyword(evaluation_string,"var")
payload['Contains Eval']=cal_keyword(evaluation_string,"eval")
payload['Contains href']=cal_keyword(evaluation_string,"href")
payload['Contains Cookie']=cal_keyword(evaluation_string,"cookie")
payload['Contains StringfromCharCode']=cal_keyword(evaluation_string,"string.fromcharcode")
payload['Contains Single Quote']=cal_keyword(evaluation_string,"'")
payload['Contains Question Mark']=cal_keyword(evaluation_string,"?")
payload['Contains Exclamation Mark']=cal_keyword(evaluation_string,"!")
payload['Contains Semicolon']=cal_keyword(evaluation_string,";")
payload['Contains HTTP']=cal_keyword(evaluation_string,"http")
payload['Contains JS']=cal_keyword(evaluation_string,"jsp")
payload['Contains Hash']=cal_keyword(evaluation_string,"#")
payload['Contains Equal']=cal_keyword(evaluation_string,"=")
payload['Contains Open Bracket']=cal_keyword(evaluation_string,"[")
payload['Contains Close Bracket']=cal_keyword(evaluation_string,"]")
payload['Contains Duble Bracket']=cal_keyword(evaluation_string,"[") * cal_keyword(evaluation_string,"]")
payload['Contains Dollar']=cal_keyword(evaluation_string,"$")
payload['Contains Open Parenthesis']=cal_keyword(evaluation_string,"(")
payload['Contains Close Parenthesis']=cal_keyword(evaluation_string,"(")
payload['Contains Asterik']=cal_keyword(evaluation_string,"*")
payload['Contains Comma']=cal_keyword(evaluation_string,",")
payload['Contains Hyphen']=cal_keyword(evaluation_string,"-")
payload['Contains Less Than']=cal_keyword(evaluation_string,"<")
payload['Contains Greater Than']=cal_keyword(evaluation_string,">")
payload['Contains At']=cal_keyword(evaluation_string,"@")
payload['Contains Underscore']=cal_keyword(evaluation_string,"_")
payload['Contains Location']=cal_keyword(evaluation_string,"location")
payload['Contains Search']=cal_keyword(evaluation_string,"search")
payload['Contains &#']=cal_keyword(evaluation_string,"&#")
payload['Contains Colon']=cal_keyword(evaluation_string,":")
payload['Contains Dots']=cal_keyword(evaluation_string,".")
payload['Contains Open Brace']=cal_keyword(evaluation_string,"{")
payload['Contains Close Brace']=cal_keyword(evaluation_string,"}")
payload['Contains tilde']=cal_keyword(evaluation_string,"~")
payload['Contains Spase']=cal_keyword(evaluation_string," ")
payload['Contains Qution']=cal_keyword(evaluation_string,"\"")
payload['Contains Grave']=0
payload['Contains Duble Equals']=cal_keyword(evaluation_string,"==")
payload['Contains Duble Slash']=cal_keyword(evaluation_string,"//")
payload['Contains Vertical Bar']=cal_keyword(evaluation_string,"|")
payload['Contains Power']=cal_keyword(evaluation_string,"^")
payload['Contains Broken Bar']=cal_keyword(evaluation_string,"¦")
payload['Contains Alert']=cal_keyword(evaluation_string,"alert")
payload['Contains Break Line']=cal_keyword(evaluation_string,"br")
payload['Letters Ratio']=format_float = "{:.2f}".format(letter_ratio(evaluation_string))
payload['Numbuer Ratio']=format_float = "{:.2f}".format(number_ratio(evaluation_string))
payload['Symbols Ratio']=format_float = "{:.2f}".format(symbol_ratio(evaluation_string))
payload['Class']=0
payload=pd.DataFrame([payload])
payload.head
X = np.asarray(payload.drop(labels=['Class'],axis=1)).reshape(len(payload),1,65).astype('float32')

y = np.asarray(payload['Class']).astype('float32')


test_payload = tf.data.Dataset.from_tensor_slices((X, y))
test_payload = test_payload.batch(BATCH_SIZE)
pred = new_model.predict(test_payload)[0][0][0]
print(pred)




0.99551344
